### Stage 2. Feature engineering for the credit scoring model.
| Field                                             | Keep/Remove         | Rationale                                                      |
|---------------------------------------------------|---------------------|----------------------------------------------------------------|
| purpose, home_ownership                           | Keep                | Important contextual factors                                   |
| hardship_flag, hardship_loan_status, hardship_dpd | Combine ?           | Considering combining into single hardship indicator           |
| emp_length, earliest_cr_line                      | Keep                | Important stability indicators                                 |
| loan_amnt, funded_amnt, installment               | Keep                | Core loan characteristics that directly impact risk assessment |
| fico_range_low, fico_range_high                   | Keep                | Critical credit quality indicators                             |
| term, int_rate, grade, sub_grade                  | Keep                | Core loan terms and risk assessment                            |
| delinq_2yrs, mths_since_last_delinq               | Keep                | Key delinquency metrics                                        |
| open_acc, total_acc, open_acc_6m                  | Keep                | Account management indicators                                  |
| is_joint_app                                      | Keep                | Important application characteristic                           |
| inq_last_6mths                                    | Keep                | Recent credit seeking behavior                                 |
| pub_rec, mths_since_last_record                   | Keep                | Public record items important for risk                         |
| mths_since_last_major_derog                       | Keep                | Important derogatory indicator                                 |
| annual_inc, annual_inc_joint                      | Keep one            | Individual income for individual and joint applications        |
| dti, dti_joint                                    | Keep one            | Same logic as income                                           |
| revol_util, revol_bal, revol_bal_joint            | Keep one            | Keep individual revolving data for non-joint apps              |
| initial_list_status                               | Remove?             | Less relevant for risk assessment                              |
| sec_app_earliest_cr_line                          | Remove if non-joint | Only relevant for joint applications                           |

### Engineering the `home_ownership` fields with `create_home_ownership_ordinal()` in /feature_engineering/nodes.py
Functionality:
Creates an ordinal encoded feature home_ownership_ordinal from home_ownership, reflecting risk levels.
Input: home_ownership — categorical field with values like "own", "mortgage", "rent", "other"
Output: `home_ownership_ordinal` — integer feature (0 = safest, 3 = riskiest)
New field(s): `home_ownership_ordinal` — used in both tree-based and regression models.
Drop field(s): `home_ownership` -- used only to create home_ownership_ordinal

In [35]:
# cols_for_reg = []  # columns for the regression-based models
# cols_for_tree = []  # columns for thew tree-based models
drop_cols = ["home_ownership"] # columns to drop entirely

# print(f"Columns for the regression-based models: {cols_for_reg}")
# print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop: {drop_cols}")

Columns to drop ['home_ownership']


### Engineering the `purpose` field with `encode_purpose_field()` in /feature_engineering/nodes.py

**Functionality:**
Generates one-hot encoded features from a cleaned version of the `purpose` field (`purpose_cleaned`), replacing rare categories with `"other"`.
Useful primarily for **tree-based models**, which can leverage one-hot encoded categorical splits.

**Input:**  - `purpose` — a preprocessed categorical field (lowercase, space-normalized)

**Intermediate Output:**  - `purpose_cleaned` — categorical field where rare categories are grouped as `"other"`

**Output:**
- One-hot encoded fields:
  - `purpose_<value>` for each cleaned purpose category
  - Includes `purpose_other` and `purpose_nan`

**New field(s):**
- `purpose_cleaned` — used to generate the one-hot columns
- `purpose_*` — one-hot encoded columns, **tree model-specific**

**Drop field(s):**
- `purpose` — used only for generating `purpose_cleaned` and will be used in `create_loan_purpose_risk_features()` below
- `purpose_cleaned` — optional to drop after one-hot encoding, unless used later

> 💡 Tip: Keep `purpose_cleaned` if any downstream analysis requires interpretable categorical grouping. Otherwise, it can be dropped post one-hot encoding.


In [36]:

cols_for_tree = ["purpose_other", "purpose_nan"]


drop_cols = list(set(drop_cols + ["purpose_cleaned"]))

# print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the tree-based models: ['purpose_other', 'purpose_nan']
Columns to drop ['home_ownership', 'purpose_cleaned']


### Engineering the `purpose` fields with `create_loan_purpose_risk_features()` in /feature_engineering/nodes.py
Functionality:
Transforms the purpose field into multiple risk-based features using business logic (risk mappings) and creates derived features useful for both tree-based and regression models.
Input: `purpose` -- a preprocessed categorical feature (e.g., "debt consolidation", "medical")
Output_1: `purpose_risk_score` — integer score from 1 (low risk) to 4 (very high risk) -- Regression + Tree
Output_2: `purpose_risk_category` —  categorical label (low, medium, etc.) -- Tree only (used in one-hot encoding or tree splits)
Output_3: `purpose_high_risk` — binary indicator (1 if risk_score ≥ 3) -- tree only
New field(s): `purpose_risk_score` — used in tree and regression models.
New field(s): `purpose_risk_category`, `purpose_high_risk` — used in tree-based models.
Drop field(s): `purpose` —  used to generate all 3 output columns above

In [37]:
cols_for_reg = ["home_ownership_ordinal", "purpose_risk_score"]

cols_for_tree = list(set(cols_for_tree + ["home_ownership_ordinal", "purpose_risk_score", "purpose_risk_category", "purpose_high_risk"]))

drop_cols = list(set(drop_cols + ["purpose"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['home_ownership_ordinal', 'purpose_risk_score']
Columns for the tree-based models: ['purpose_nan', 'home_ownership_ordinal', 'purpose_risk_category', 'purpose_risk_score', 'purpose_high_risk', 'purpose_other']
Columns to drop ['home_ownership', 'purpose_cleaned', 'purpose']


### Engineering the `hardship_flag` field with `create_has_hardship_flag()` in /feature_engineering/nodes.py

**Functionality:**
Converts the categorical `hardship_flag` column (values like `'y'` / `'n'`) into a binary numeric indicator `has_hardship`.

**Input:**
- `hardship_flag` — a string column indicating if the borrower had a hardship (values like `"y"`, `"n"`, case-insensitive)

**Output:**
- `has_hardship` — binary indicator:
  - `1` if hardship_flag is `"y"`
  - `0` otherwise

**New field(s):**
- `has_hardship` — **used in both tree-based and regression models**

**Drop field(s):**
- `hardship_flag` — only used for generating `has_hardship`, can be dropped after this node


In [38]:
cols_for_reg = list(set(cols_for_reg + ["has_hardship"]))

cols_for_tree = list(set(cols_for_tree + ["has_hardship"]))

drop_cols = list(set(drop_cols + ["hardship_flag"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['home_ownership_ordinal', 'purpose_risk_score', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'home_ownership_ordinal', 'purpose_risk_category', 'purpose_risk_score', 'purpose_high_risk', 'purpose_other', 'has_hardship']
Columns to drop ['home_ownership', 'purpose_cleaned', 'purpose', 'hardship_flag']


### Engineering the `hardship_loan_status` field with `create_was_late_before_hardship()` in /feature_engineering/nodes.py

**Functionality:**
Creates a binary feature that indicates whether a borrower entered hardship while already in a **late loan status**.

**Input:**
- `hardship_loan_status` — string field indicating loan status at time of hardship (e.g., `"late (31-120 days)"`, `"fully paid"`, etc.)

**Output:**
- `was_late_before_hardship` — binary indicator:
  - `1` if the loan status at hardship contains the word `"late"` (case-insensitive)
  - `0` otherwise or if field is missing

**New field(s):**
- `was_late_before_hardship` — **used in both tree-based and regression models**

**Drop field(s):**
- `hardship_loan_status` — used only to derive `was_late_before_hardship`, can be dropped after this node


In [39]:
cols_for_reg = list(set(cols_for_reg + ["was_late_before_hardship"]))

cols_for_tree = list(set(cols_for_tree + ["was_late_before_hardship"]))

drop_cols = list(set(drop_cols + ["hardship_loan_status"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'purpose_risk_score', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'home_ownership_ordinal', 'purpose_risk_category', 'purpose_risk_score', 'purpose_high_risk', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'purpose', 'hardship_flag', 'home_ownership', 'hardship_loan_status']


### Engineering the `hardship_dpd` field with `create_hardship_features()` in /feature_engineering/nodes.py

**Functionality:**
Fills missing values in the `hardship_dpd` (days past due during hardship) field and creates a cleaned version for modeling.

**Input:**
- `hardship_dpd` — numeric field representing how many days the loan was past due at the time of hardship (can be NaN)

**Output:**
- `hardship_dpd_filled` — numeric field with missing values filled as `0`

**New field(s):**
- `hardship_dpd_filled` — **used in both tree-based and regression models**

**Drop field(s):**
- `hardship_dpd` — original raw column used only to create the cleaned version, can be dropped after this node


In [40]:
cols_for_reg = list(set(cols_for_reg + ["hardship_dpd_filled"]))

cols_for_tree = list(set(cols_for_tree + ["hardship_dpd_filled"]))

drop_cols = list(set(drop_cols + ["hardship_dpd"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'purpose_risk_score', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'purpose_risk_category', 'purpose_risk_score', 'purpose_high_risk', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'purpose', 'hardship_flag', 'home_ownership', 'hardship_loan_status', 'hardship_dpd']


### Engineering the `emp_length` field with `engineer_emp_length_features()` in /feature_engineering/nodes.py

**Functionality:**
Transforms the text-based `emp_length` field into numeric versions for use in both tree-based and regression models. Handles missing values separately depending on the model type.

**Input:**
- `emp_length` — a categorical field representing the number of years employed (e.g., `"10+ years"`, `"< 1 year"`)

**Output:**
- `emp_length_clean` — numeric base transformation (used internally)
- `emp_length_clean_tree` — tree-model-specific version with missing filled as `-1`
- `emp_length_clean_reg` — regression-model-specific version with missing filled as **median**

**New field(s):**
- `emp_length_clean_tree` — **used in tree-based models**
- `emp_length_clean_reg` — **used in regression models**

**Drop field(s):**
- `emp_length` — original text field, can be dropped after this node
- `emp_length_clean` — intermediate transformation, not directly used downstream


In [41]:
cols_for_reg = list(set(cols_for_reg + ["emp_length_clean_reg"]))

cols_for_tree = list(set(cols_for_tree + ["emp_length_clean_tree"]))

drop_cols = list(set(drop_cols + ["emp_length", "emp_length_clean"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'purpose_risk_score', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'purpose_risk_category', 'purpose_risk_score', 'emp_length_clean_tree', 'purpose_high_risk', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'purpose', 'hardship_flag', 'emp_length', 'home_ownership', 'hardship_loan_status', 'emp_length_clean', 'hardship_dpd']


### Engineering the `earliest_cr_line` fields with `create_credit_age_feature()` in /feature_engineering/nodes.py

**Functionality:**
Combines individual and joint applicants’ credit line information into a unified field and calculates credit history length (age) in months.

**Inputs:**
- `earliest_cr_line` — datetime field for the primary applicant’s earliest credit line
- `sec_app_earliest_cr_line` — datetime for secondary applicant (if joint)
- `issue_d` — loan issue date
- `is_joint_app` — binary flag indicating joint application

**Outputs:**
- `earliest_cr_line_final` — unified field (uses secondary applicant’s date if applicable)
- `credit_age_months` — numeric field representing months between credit line and loan issue

**New field(s):**
- `earliest_cr_line_final` — used as temporal reference, especially for joint cases
- `credit_age_months` — used in **both** tree-based and regression models

**Drop field(s):**
- `earliest_cr_line` — superseded by `earliest_cr_line_final`
- `sec_app_earliest_cr_line` — no longer needed after merge
- `issue_d` — used in other features too, so **retain**


In [42]:
cols_for_reg = list(set(cols_for_reg + ["credit_age_months"]))

cols_for_tree = list(set(cols_for_tree + ["credit_age_months"]))

drop_cols = list(set(drop_cols + ["earliest_cr_line", "sec_app_earliest_cr_line", "earliest_cr_line_final"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_score', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_category', 'purpose_risk_score', 'emp_length_clean_tree', 'purpose_high_risk', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'purpose', 'hardship_flag', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_status', 'emp_length_clean', 'hardship_dpd', 'earliest_cr_line', 'earliest_cr_line_final']


### Engineering the `loan_amnt` and `installment` fields with `create_loan_amount_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Creates ratio-based and binned features that capture the relationship between loan amount and monthly installment. These features reflect affordability and general loan size.

**Input Fields:**
- `loan_amnt` – total loan amount
- `installment` – monthly payment amount

**Output Fields:**
- `loan_to_installment_ratio` – ratio of loan amount to installment payment, clipped to 1st–99th percentile. Used in **tree-based and regression** models
- `loan_amount_band` – decile-based binning of `loan_amnt` (0–9), fallback to fixed bins if needed. Used in **tree-based** models (categorical splits)

**Drop Field(s):**
- _None dropped at this stage_
  `loan_amnt` and `installment` are still used in:
  - `income_to_loan_reg`
  - `payment_to_income`
  - Downstream validation and feature profiling

⚠️ _Dropping `loan_amnt` and `installment` after all dependent features are created._


### The `funded_amnt` field will be deleted since its correlated with `loan_amnt`

In [43]:
cols_for_reg = list(set(cols_for_reg + ["loan_to_installment_ratio"]))

cols_for_tree = list(set(cols_for_tree + ["loan_to_installment_ratio", "loan_amount_band"]))

drop_cols = list(set(drop_cols + ["funded_amnt"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_score', 'loan_to_installment_ratio', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_category', 'purpose_risk_score', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'purpose_high_risk', 'loan_amount_band', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'purpose', 'hardship_flag', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_status', 'funded_amnt', 'emp_length_clean', 'hardship_dpd', 'earliest_cr_line', 'earliest_cr_line_final']


### Engineering the `fico` fields with `create_fico_score_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Generates standardized and model-friendly FICO score features by averaging FICO bounds and creating binned risk indicators.

**Inputs:**
- `fico_range_low`: Lower end of the borrower's estimated FICO range
- `fico_range_high`: Upper end of the borrower's estimated FICO range

**Output Fields:**
- `fico_average`: Mean of `fico_range_low` and `fico_range_high`, clipped to [300–850]
  - Used in **tree-based** and **regression** models
- `fico_risk_band`: Categorical score band (A–F), using either quantiles or fallback fixed bins
  - Used in **tree-based** models only

**New Field(s):**
- `fico_average` —  Tree & Regression
- `fico_risk_band` —  Tree only

**Drop Field(s):**
- `fico_range_low` —  Drop after creating `fico_average`
- `fico_range_high` —  Drop after creating `fico_average`


In [44]:
cols_for_reg = list(set(cols_for_reg + ["fico_average"]))

cols_for_tree = list(set(cols_for_tree + ["fico_average", "fico_risk_band"]))

drop_cols = list(set(drop_cols + ["fico_range_low", "fico_range_high"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_score', 'loan_to_installment_ratio', 'fico_average', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'fico_risk_band', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_category', 'purpose_risk_score', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'purpose_high_risk', 'fico_average', 'loan_amount_band', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_low', 'purpose', 'hardship_flag', 'fico_range_high', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_status', 'funded_amnt', 'emp_length_clean', 'hardship_dpd', 'earliest_cr_line', 'earliest_cr_line_final']


### Engineering the `term` fields with `create_term_model_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Generates model-specific features from the `term` column (e.g., 36 or 60 months) for both tree-based and regression models.

**Input:**
- `term` — numeric loan term in months (already cleaned during preprocessing)
- `int_rate` — interest rate ( is used later)

**Outputs:**
- `term_tree` — Categorical feature used in tree-based models -- Tree-only
- `term_normalized_reg` — Scaled feature (0–1) for regression models  -- Regression-only
- `term_rate_interaction_reg` — Interaction between term and interest rate -- Regression-only

**New field(s):**
- `term_tree`
- `term_normalized_reg`
- `term_rate_interaction_reg`

**Drop field(s):**
- `term` — can be dropped **after** derived features are created, as it is replaced by model-specific versions.


In [45]:
cols_for_reg = list(set(cols_for_reg + ["term_normalized_reg", "term_rate_interaction_reg"]))

cols_for_tree = list(set(cols_for_tree + ["term_tree"]))

drop_cols = list(set(drop_cols + ["term"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_score', 'loan_to_installment_ratio', 'fico_average', 'term_rate_interaction_reg', 'term_normalized_reg', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'fico_risk_band', 'term_tree', 'was_late_before_hardship', 'home_ownership_ordinal', 'hardship_dpd_filled', 'credit_age_months', 'purpose_risk_category', 'purpose_risk_score', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'purpose_high_risk', 'fico_average', 'loan_amount_band', 'purpose_other', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_low', 'fico_range_high', 'purpose', 'hardship_flag', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_status', 'funded_amnt', 'emp_length_clean', 'hardship_dpd', 'earliest_cr_line', 'term', 'earliest_cr_line_final']


### Engineering the `int_rate`, `grade`, and `sub_grade` fields with `encode_interest_and_grade_fields()` in `/feature_engineering/nodes.py`

**Functionality:**
Transforms key LendingClub classification and interest rate fields into numerical features for modeling.

**Inputs:**
- `int_rate` — interest rate in percentage (e.g., 13.49)
- `grade` — letter grade from A to G (already lowercased in preprocessing)
- `sub_grade` — finer-grained version of `grade`, from A1 to G5 (e.g., "b3", "f2")

**Outputs:**
- `int_rate` — converted from percentage to decimal (e.g., 0.1349) ✅ Regression + Tree
- `grade_encoded` — ordinal feature (1 = best grade A, 7 = worst grade G) ✅ Regression + Tree
- `sub_grade_encoded` — fine-grained ordinal feature (1 = A1, ..., 35 = G5) ✅ Regression + Tree

**New field(s):**
- `grade_encoded`
- `sub_grade_encoded`
- (transformed) `int_rate` (overwrites original)

**Drop field(s):**
- `grade` — used to create `grade_encoded`
- `sub_grade` — used to create `sub_grade_encoded`
- ⚠️ Retain `int_rate` (overwritten with numeric format)


In [46]:
cols_for_reg = list(set(cols_for_reg + ["int_rate", "grade_encoded", "sub_grade_encoded"]))

cols_for_tree = list(set(cols_for_tree + ["int_rate", "grade_encoded", "sub_grade_encoded"]))

drop_cols = list(set(drop_cols + ["grade", "sub_grade"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'grade_encoded', 'sub_grade_encoded', 'purpose_risk_score', 'loan_to_installment_ratio', 'fico_average', 'int_rate', 'term_normalized_reg', 'term_rate_interaction_reg', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'fico_risk_band', 'home_ownership_ordinal', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'loan_amount_band', 'purpose_other', 'term_tree', 'credit_age_months', 'fico_average', 'grade_encoded', 'purpose_high_risk', 'int_rate', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_low', 'fico_range_high', 'purpose', 'hardship_flag', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_status', 'grade', 'funded_amnt', 'emp_length_clean', '

### Engineering credit history indicators with `create_credit_history_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Combines multiple credit history fields into composite features that signal borrower risk due to past derogatory marks or delinquencies.

**Inputs:**
- `delinq_2yrs` — number of delinquencies in the past 2 years
- `pub_rec` — number of derogatory public records
- `mths_since_last_delinq` — months since the last delinquency (may contain NaNs)

**Outputs:**
- `has_derogatory` — binary indicator (1 if `delinq_2yrs` > 0 or `pub_rec` > 0) ✅ Tree + Regression
- `delinq_weight` — weighted delinquency severity score (higher if delinquencies were recent) ✅ Regression only

**New field(s):**
- `has_derogatory` — used in both tree-based and regression models
- `delinq_weight` — used in regression models only (continuous score)

**Drop field(s):**
- None explicitly dropped yet, but:
  - `delinq_2yrs`, `pub_rec`, and `mths_since_last_delinq` keep temporarily for use in later features (`create_credit_history_model_features`)


In [47]:
cols_for_reg = list(set(cols_for_reg + ["has_derogatory", "delinq_weight"]))

cols_for_tree = list(set(cols_for_tree + ["has_derogatory"]))

# drop_cols = list(set(drop_cols + ["grade", "sub_grade"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'delinq_weight', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'sub_grade_encoded', 'purpose_risk_score', 'loan_to_installment_ratio', 'grade_encoded', 'fico_average', 'int_rate', 'term_rate_interaction_reg', 'has_derogatory', 'term_normalized_reg', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'has_derogatory', 'fico_risk_band', 'home_ownership_ordinal', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'loan_amount_band', 'purpose_other', 'term_tree', 'credit_age_months', 'fico_average', 'grade_encoded', 'purpose_high_risk', 'int_rate', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_low', 'fico_range_high', 'purpose', 'hardship_flag', 'emp_length', 'sec_app_earliest_cr_line', 'home_ownership', 'hardship_loan_s

### Engineering model-specific credit history features with `create_credit_history_model_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Generates separate credit history features for tree-based and regression models by handling missing values and emphasizing recent delinquencies.

**Inputs:**
- `delinq_2yrs` — number of delinquencies in the past 2 years
- `mths_since_last_delinq` — months since the last delinquency

**Outputs:**

✅ **Tree-based model features**
- `delinq_2yrs_tree` — `delinq_2yrs` with missing values filled by `-1`
- `has_delinq_tree` — binary indicator for any delinquency
- `has_recent_delinq_tree` — binary flag for recent delinquency (< 24 months)

✅ **Regression model features**
- `delinq_2yrs_reg` — `delinq_2yrs` with median imputation
- `delinq_severity_reg` — delinquency severity (higher if recent)

**New field(s):**
- Tree models: `delinq_2yrs_tree`, `has_delinq_tree`, `has_recent_delinq_tree`
- Regression models: `delinq_2yrs_reg`, `delinq_severity_reg`

**Drop field(s):**
- None dropped explicitly
- `delinq_2yrs` and `mths_since_last_delinq` may be dropped later after final model-specific datasets are built


In [48]:
cols_for_reg = list(set(cols_for_reg + ["delinq_2yrs_reg", "delinq_severity_reg"]))

cols_for_tree = list(set(cols_for_tree + ["delinq_2yrs_tree", "has_delinq_tree", "has_recent_delinq_tree"]))

# drop_cols = list(set(drop_cols + ["grade", "sub_grade"]))
# TODO: check other functions for `delinq_2yrs` and `mths_since_last_delinq`

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'home_ownership_ordinal', 'delinq_weight', 'emp_length_clean_reg', 'hardship_dpd_filled', 'credit_age_months', 'delinq_2yrs_reg', 'sub_grade_encoded', 'delinq_severity_reg', 'purpose_risk_score', 'loan_to_installment_ratio', 'grade_encoded', 'fico_average', 'int_rate', 'term_normalized_reg', 'has_derogatory', 'term_rate_interaction_reg', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'has_delinq_tree', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'has_derogatory', 'has_recent_delinq_tree', 'fico_risk_band', 'home_ownership_ordinal', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_tree', 'loan_amount_band', 'purpose_other', 'term_tree', 'credit_age_months', 'fico_average', 'grade_encoded', 'purpose_high_risk', 'int_rate', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_low', 'fico_range_high

### Handling missing `pub_rec` values with `handle_pub_rec_missing()` in `/feature_engineering/nodes.py`

**Functionality:**
Fills missing values in the `pub_rec` field using a specified imputation strategy.

**Inputs:**
- `pub_rec` — public derogatory record count
- `strategy` — either `"median"` or `"negative_one"` (default is `"median"`)

**Outputs:**
- `pub_rec` — updated with imputed values

**New field(s):**
- None created — the original `pub_rec` field is modified in-place

**Drop field(s):**
- None

**Model usage:**
- `pub_rec` is used by **both tree-based** and **regression** models.
  - Tree: as-is or through features like `has_derogatory`
  - Regression: often used in engineered ratios or severity scores

**Notes:**
- The strategy is configurable through pipeline parameters (`params:pub_rec_strategy`)
- This node ensures that later modeling steps can safely rely on non-missing `pub_rec`


In [49]:
cols_for_reg = list(set(cols_for_reg + ["pub_rec"]))

cols_for_tree = list(set(cols_for_tree + ["pub_rec"]))

# drop_cols = list(set(drop_cols + ["grade", "sub_grade"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'term_normalized_reg', 'emp_length_clean_reg', 'delinq_severity_reg', 'has_derogatory', 'home_ownership_ordinal', 'delinq_weight', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_reg', 'term_rate_interaction_reg', 'credit_age_months', 'fico_average', 'grade_encoded', 'int_rate', 'pub_rec', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'has_delinq_tree', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'has_derogatory', 'has_recent_delinq_tree', 'fico_risk_band', 'home_ownership_ordinal', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_tree', 'loan_amount_band', 'purpose_other', 'term_tree', 'credit_age_months', 'fico_average', 'grade_encoded', 'purpose_high_risk', 'int_rate', 'pub_rec', 'has_hardship']
Columns to drop ['purpose_cleaned', 'fico_range_

### Engineering delinquency features with `create_delinquency_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Generates composite indicators of borrower delinquency behavior using recent and historical fields.

**Inputs:**
- `delinq_2yrs`: Number of delinquencies in the past 2 years
- `mths_since_last_delinq`: Months since last delinquency
- `acc_now_delinq`: Number of accounts currently delinquent
- `mths_since_recent_revol_delinq`: (present but not used in current logic)

**Outputs:**
- `delinquency_score` — composite feature capturing overall delinquency risk
- `recent_delinq_bin` — ordinal bin of `mths_since_last_delinq` (used to adjust score)

**New field(s):**
- `delinquency_score` — used in **tree-based models** (intended for splits or ranking risk)
- `recent_delinq_bin` — tree-only ordinal bucket useful for boosting models

**Drop field(s):**
- `delinq_2yrs` and `mths_since_last_delinq` - to add to the drop list and the end of the feature creation

**Model usage:**
- `delinquency_score` and `recent_delinq_bin` are intended primarily for **tree-based models**
- Raw fields like `delinq_2yrs` and `acc_now_delinq` are retained for **model-specific encoding**

**Notes:**
- Combines multiple weak indicators into a stronger aggregate risk metric
- Supports interpretability in feature importance plots
- Score is clipped to a max of 10 to avoid dominance in feature scales


In [51]:
# cols_for_reg = list(set(cols_for_reg + ["pub_rec"]))

cols_for_tree = list(set(cols_for_tree + ["delinquency_score", "recent_delinq_bin"]))

drop_cols = list(set(drop_cols + ["delinq_2yrs", "mths_since_last_delinq", "mths_since_recent_revol_delinq", "acc_now_delinq"]))

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'term_normalized_reg', 'emp_length_clean_reg', 'delinq_severity_reg', 'has_derogatory', 'home_ownership_ordinal', 'delinq_weight', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_reg', 'term_rate_interaction_reg', 'credit_age_months', 'fico_average', 'grade_encoded', 'int_rate', 'pub_rec', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'has_delinq_tree', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'delinquency_score', 'has_derogatory', 'has_recent_delinq_tree', 'fico_risk_band', 'home_ownership_ordinal', 'recent_delinq_bin', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_tree', 'loan_amount_band', 'purpose_other', 'term_tree', 'credit_age_months', 'fico_average', 'grade_encoded', 'purpose_high_risk', 'int_rate', 'pub_rec', 'has_hardship']
Column

### Engineering credit inquiry features with `create_credit_inquiry_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Creates new features that quantify recent credit inquiry behavior and potential risk associated with frequent credit seeking.

**Inputs:**
- `inq_last_6mths` — number of inquiries in the last 6 months
- `inq_last_12m` — number of inquiries in the last 12 months
- `open_acc_6m` — number of accounts opened in the last 6 months

**Outputs:**
- `recent_inquiry_intensity` — ratio of inquiries in 6m vs 12m (clipped to 1) — **Regression + Tree**
- `high_recent_inquiries` — binary flag for frequent inquiries (3 or more in last 6m) — **Tree**
- `inq_to_open_acc_ratio` — ratio of inquiries to opened accounts (proxy for credit seeking success rate) — **Regression + Tree**

**New field(s):**
- `recent_inquiry_intensity` — normalized ratio — **used in regression models**
- `high_recent_inquiries` — binary risk flag — **used in tree models**
- `inq_to_open_acc_ratio` — success ratio of inquiries — **used in both models**

**Drop field(s):**
- None (original input fields may still be useful elsewhere)


In [52]:
cols_for_reg = list(set(cols_for_reg + ["recent_inquiry_intensity", "inq_to_open_acc_ratio"]))

cols_for_tree = list(set(cols_for_tree + ["recent_inquiry_intensity", "high_recent_inquiries", "inq_to_open_acc_ratio"]))

# drop_cols = list(set(drop_cols + []))
# TODO: evaluate input variables `inq_last_6mths`, `inq_last_12m` and `open_acc_6m` for the rest of functions

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")

Columns for the regression-based models: ['was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'term_normalized_reg', 'emp_length_clean_reg', 'delinq_severity_reg', 'inq_to_open_acc_ratio', 'has_derogatory', 'recent_inquiry_intensity', 'home_ownership_ordinal', 'delinq_weight', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_reg', 'term_rate_interaction_reg', 'credit_age_months', 'fico_average', 'grade_encoded', 'int_rate', 'pub_rec', 'has_hardship']
Columns for the tree-based models: ['purpose_nan', 'has_delinq_tree', 'was_late_before_hardship', 'hardship_dpd_filled', 'loan_to_installment_ratio', 'emp_length_clean_tree', 'high_recent_inquiries', 'delinquency_score', 'inq_to_open_acc_ratio', 'has_derogatory', 'has_recent_delinq_tree', 'recent_inquiry_intensity', 'fico_risk_band', 'home_ownership_ordinal', 'recent_delinq_bin', 'purpose_risk_category', 'sub_grade_encoded', 'purpose_risk_score', 'delinq_2yrs_tree', 'loan_amount_band', 'purpose_other', 't

### Engineering account activity features with `create_account_activity_features()` in `/feature_engineering/nodes.py`

**Functionality:**
Generates features that describe the borrower’s recent credit account behavior, such as how quickly they are acquiring new credit and how active their accounts are.

**Inputs:**
- `open_acc` — number of currently open accounts
- `acc_open_past_24mths` — number of accounts opened in the last 24 months
- `total_acc` — total number of credit accounts

**Outputs:**
- `recent_acc_ratio` — share of open accounts that are newly opened
- `rapid_acc_acquisition` — binary flag (1 if more than 50% of open accounts were opened recently)
- `active_acc_ratio` — proportion of total accounts that are still open

**New field(s):**
- `recent_acc_ratio` — **used in tree and regression models**
- `rapid_acc_acquisition` — **used in tree-based models**
- `active_acc_ratio` — **used in tree and regression models**

**Drop field(s):**
- `open_acc`, `acc_open_past_24mths`, `total_acc` may be dropped after model-specific features are finalized if no longer used downstream

**Model usage:**
- `recent_acc_ratio`, `active_acc_ratio` — useful for both **regression** and **tree-based** models
- `rapid_acc_acquisition` — binary feature primarily for **tree-based** models


In [ ]:
cols_for_reg = list(set(cols_for_reg + ["recent_acc_ratio", "active_acc_ratio"]))

cols_for_tree = list(set(cols_for_tree + ["recent_acc_ratio", "rapid_acc_acquisition", "active_acc_ratio"]))

# drop_cols = list(set(drop_cols + []))
# TODO: evaluate input variables `open_acc`, `acc_open_past_24mths` and `total_acc` for the rest of functions

print(f"Columns for the regression-based models: {cols_for_reg}")
print(f"Columns for the tree-based models: {cols_for_tree}")
print(f"Columns to drop {drop_cols}")